In [1]:
import evadb

TASK_TABLE_NAME = 'task'

In [3]:
cursor = evadb.connect().cursor()
try:
    cursor.query("""
        CREATE DATABASE mydb WITH ENGINE = 'sqlite', PARAMETERS = {
        "database": "eva.db"
    };""").df()
except:
    pass

# drop all tables
cursor.query(f"""
    DROP TABLE IF EXISTS {TASK_TABLE_NAME};
""").df()
cursor.query("""
use mydb {
    DROP TABLE IF EXISTS members
};
""").df()

# Create tables
cursor.query(f"""
    CREATE TABLE {TASK_TABLE_NAME} (
        id INTEGER UNIQUE,
        name TEXT(30),
        description	TEXT(100)
    );
""").df()
# cursor.query("""
# use mydb {
#     CREATE TABLE "members" (
#         "id"	INTEGER NOT NULL UNIQUE,
#         "name"	TEXT
#     )
# };
# """).df()


10-17-2023 08:15:36 ERROR [plan_executor:plan_executor.py:execute_plan:0182] mydb already exists.
Traceback (most recent call last):
  File "c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\executor\plan_executor.py", line 178, in execute_plan
    yield from output
  File "c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\executor\create_database_executor.py", line 42, in exec
    raise ExecutorError(f"{self.node.database_name} already exists.")
evadb.executor.executor_utils.ExecutorError: mydb already exists.


""


In [81]:
# list tables
cursor.query("""
use mydb {
   SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%'
};
""").df()

,name
0,tasks


In [82]:
# create new task
cursor.query(f"""
        INSERT INTO {TASK_TABLE_NAME} (id, name, description) VALUES (1, 'test name', 'test description');
    """).df()

x = cursor.query("""
    SELECT * FROM mydb.tasks;  
    """).df()
print(x)

   tasks.id tasks.name tasks.description
0         1  test name  test description


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(
